In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.applications import MobileNetV2 # type: ignore
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint # type: ignore

2024-05-30 15:05:33.176687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Paths
train_data_dir = "dataset/training"
test_data_dir = "dataset/test"
wip_models_dir = "models_wip"
models_dir = "models"
model_name = "chess_classifier_10k"
current_best = os.path.join(wip_models_dir, f"{model_name}_best.keras")
final = os.path.join(models_dir, f"{model_name}.keras")

# Parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 10
num_classes = 13  # 6 pieces x 2 colors + 1 empty

In [3]:
os.makedirs(wip_models_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

In [4]:
# Data normalization
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [5]:
# Train and validation generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)

Found 32000 images belonging to 13 classes.
Found 8000 images belonging to 13 classes.


In [6]:
# Calculate steps_per_epoch and validation_steps
# steps_per_epoch = max(1, train_generator.samples // batch_size)
# validation_steps = max(1, validation_generator.samples // batch_size)
steps_per_epoch = max(1, len(train_generator) // batch_size)
validation_steps = max(1, len(validation_generator) // batch_size)

## Creating the model

Only using a single dense layer on top of MobileNet's, could probably improve the model by adding a few more.

In [7]:
# Load pre-trained MobileNetV2 model + higher level layers
base_model = MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3)
)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [8]:
# Callbacks
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)
model_checkpoint = ModelCheckpoint(
    current_best, monitor="val_loss", save_best_only=True
)

## Initial training

In [9]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint],
)

Epoch 1/10


/Users/ben/Projects/chessvision/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 27s 721ms/step - accuracy: 0.6861 - loss: 1.0579 - val_accuracy: 0.9152 - val_loss: 0.1932
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 24s 770ms/step - accuracy: 0.9471 - loss: 0.1632 - val_accuracy: 0.9598 - val_loss: 0.1134
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 25s 799ms/step - accuracy: 0.9550 - loss: 0.1219 - val_accuracy: 0.9911 - val_loss: 0.0226
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 25s 828ms/step - accuracy: 0.9736 - loss: 0.0743 - val_accuracy: 0.9911 - val_loss: 0.0315
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 702ms/step - accuracy: 0.9757 - loss: 0.0569 - val_accuracy: 0.9777 - val_loss: 0.0742
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 720ms/step - accuracy: 0.9883 - loss: 0.0479 - val_accuracy: 0.9911 - val_loss: 0.0217
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 25s 798ms/step - accuracy: 0.9818 - loss: 0.0496 - val_accuracy: 0.9911 - val_loss: 0.0228
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 710ms/step - accuracy: 0.9937 - loss: 0.0210 - val_accuracy: 0.995

## Fine tuning

Only the top layers were trainable in the initial training. In order to improve predictions, we unfreeze a few ImageNet layers and retrain the model with a much lower learning rate.

This might be counter-productive, not sure, needs more testing.

Either way, the best weights are used.

In [10]:
# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Fine-tune the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint],
)

Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 39s 976ms/step - accuracy: 0.9065 - loss: 0.3298 - val_accuracy: 0.9911 - val_loss: 0.0232
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 29s 952ms/step - accuracy: 0.9633 - loss: 0.1212 - val_accuracy: 0.9866 - val_loss: 0.0502
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 29s 936ms/step - accuracy: 0.9881 - loss: 0.0543 - val_accuracy: 0.9866 - val_loss: 0.0667
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.9763 - loss: 0.0717 - val_accuracy: 0.9911 - val_loss: 0.0233
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9913 - loss: 0.0450 - val_accuracy: 0.9821 - val_loss: 0.0542
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9808 - loss: 0.0770 - val_accuracy: 0.9866 - val_loss: 0.0320
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9924 - loss: 0.0441 - val_accuracy: 0.9911 - val_loss: 0.0391
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9981 - loss: 0.0300 - val_accuracy: 1.0000 - 

In [11]:
# Save the final model
model.save(final)

In [12]:
# Evaluate the final model
loss, accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Test accuracy: {accuracy:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 735ms/step - accuracy: 1.0000 - loss: 0.0111
Test accuracy: 1.0000
